# License Plate Detection on Indian Cars using YOLOv3 and Blurring of License Plates

This repository guides you through the steps for annotating and training a custom model to detect and blur the license plates on Indian cars.

Please follow the associated article for more details - https://medium.com/@gauravsaini.728/license-plate-detection-on-indian-cars-using-yolov3-and-blurring-of-license-plates-9f2a24d48f04

# Step 1: Prepare the dataset

a. Get the images of Indian Cars with the number plate. Create 2 folders 'test' and 'train' and transfer 20% of the images in test and 80% images in train folders respectively.

Alternatively, you can use the test and train data used by me for training. Download test.zip and train.zip files from this Github Repository for accessing the annotated images.

b. Annotate the license plates in all the images in the respective test and train folders, using LabelImg, and generate the annotated .xml file for the images within the respective folders. You can get more details on LabelImg from [here
](https://github.com/tzutalin/labelImg).(Conversion process is mentioned in step 3)


# Step 2: Installing the base darknet system

In [ ]:
# Mount the Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Navigate to MyDrive
%cd /content/gdrive/MyDrive

In [ ]:
# Download YOLOv3 project
! git clone https://github.com/AlexeyAB/darknet

In [ ]:
# Install the base darknet framework
%cd darknet
! make

In [ ]:
# Uplod test and train directories in the darknet directory along with the convert.py file from this respository and run the below command
# This step will pick the coordinates from each .xml file and put them into YOLO compatible .txt file in the same test and train directories.
 # Also, train.txt and test.txt files are created within the darknet folder, containing the location of images.
 
! python convert.py

# Step 4: Prepare custom 'my_data' folder

In [20]:
# Run the below command to make a custom folder named 'my_data'
! mkdir my_data

In [21]:
# Move the train.txt and test.txt files from the darknet directory to the my_data directory.
! mv train.txt my_data/ 
! mv test.txt my_data/

In [35]:
# Create classes.names file within my_data directory with class name as "LP"
! touch /content/gdrive/MyDrive/darknet/my_data/classes.names 
! echo LP > /content/gdrive/MyDrive/darknet/my_data/classes.names

In [36]:
# Create weights directory within the my_data directory
! mkdir my_data/weights

In [37]:
# Create file darknet.data within my_data directory to provide the configuration details
! touch /content/gdrive/MyDrive/darknet/my_data/darknet.data

# Paste the below details manually in darknet.data file
## classes = 1
## train = my_data/train.txt
## valid = my_data/test.txt
## names = my_data/classes.names
## backup = my_data/weights/

In [43]:
# Copy and Paste the cfg file from darknet/cfg/yolov3.cfg to darknet/my_data directory
! cp /content/gdrive/MyDrive/darknet/cfg/yolov3.cfg /content/gdrive/MyDrive/darknet/my_data

# Make the following changes in yolov3.cfg in my_data directory.
# Line 603, 693, and 780 change the filters to 18. (filters = (classes + 5) * 3). 
#In our case we are detecting only 1 class, so the number of filters will be equal to 18.
# Line 783, change the number of classes to 1.

# **Step 5: Download the initial yolo weights for training the custom data**

In [ ]:
# Run the following command from the darknet directory
! wget https://pjreddie.com/media/files/darknet53.conv.74

## Step 6: Set criteria to save the weights file in the weights **directory**

In [ ]:
# Open detector.c file from darknet/examples directory and change line number 138 as shown below.

if(i%1000==0 || (i < 1000 && i%200 == 0))

# This change saves the weight in my_data/weights directory for every 200th iteration till 1000 iterations and then for every 1000th iteration.

## Step 7: Now start the **training**

In [ ]:
# Run the following command from the darknet directory
! ./darknet detector train /content/gdrive/MyDrive/darknet/my_data/darknet.data /content/gdrive/MyDrive/darknet/my_data/yolov3.cfg /content/gdrive/MyDrive/darknet/darknet53.conv.74

## Step 8: Detect and Blur the Licence Plate from the **image**

In [ ]:
# Change the weightsPath in line number 12 and the image file name in line number 15 before running the below code

%matplotlib inline
import numpy as np
import imutils
import cv2
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

CONF_THRESH, NMS_THRESH = 0.5, 0.5


weightsPath = 'custom/weights/yolov3_8000.weights'
configPath = 'custom/yolov3.cfg'
namesPath = 'custom/classes.names'
image = 'img3.jpeg'

# Load the network using openCV
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

# Get the output layer from YOLOv3
layers = net.getLayerNames()
output_layers = [layers[i[0] - 1] for i in net.getUnconnectedOutLayers()]


# Read and convert the image to blob and perform forward pass
img = cv2.imread(image)
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
height, width = img.shape[:2]

blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), swapRB=True, crop=False)
net.setInput(blob)
layer_outputs = net.forward(output_layers)


class_ids, confidences, b_boxes = [], [], []
for output in layer_outputs:
    for detection in output:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]

        if confidence > CONF_THRESH:
            center_x, center_y, w, h = (detection[0:4] * np.array([width, height, width, height])).astype('int')

            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            b_boxes.append([x, y, int(w), int(h)])
            confidences.append(float(confidence))
            class_ids.append(int(class_id))


# Perform non maximum suppression for the bounding boxes to filter overlapping and low confident bounding boxes
indices = cv2.dnn.NMSBoxes(b_boxes, confidences, CONF_THRESH, NMS_THRESH).flatten().tolist()

if len(indices) > 0:

    # Draw the filtered bounding boxes with their class to the image
    with open(namesPath, "r") as f:
        classes = [line.strip() for line in f.readlines()]
    colors = np.random.uniform(0, 255, size=(len(classes), 3))

    for index in indices:
        (x,y) = (b_boxes[index][0], b_boxes[index][1])
        (w,h) = (b_boxes[index][2], b_boxes[index][3])
        
        # Blur the ROI of the detected licence plate 
        img[y:y+h, x:x+w] = cv2.GaussianBlur(img[y:y+h, x:x+w] ,(35,35),0)

        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        text = "{}: {:.4f}".format("LP", confidences[index])
        cv2.putText(img, text, (x, y - 3), cv2.FONT_HERSHEY_COMPLEX_SMALL, .75 , (0, 255, 0), 1)

plt.figure(figsize=(10, 5))
plt.imshow(img)
plt.show()